In [3]:
from collections import namedtuple
import os
os.chdir("../")


In [6]:
import tensorflow as tf
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

# model_path = 'model.h5'
# model = tf.keras.models.load_model(model_path)

cap = cv2.VideoCapture(0)

# Initialize an empty array to store the landmarks
landmarks = []

# Constants for data preprocessing
face_keep_points = [0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 61, 185, 40, 39, 37]
left_hand_keep_points = [i for i in range(21)]
pose_keep_points = [0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 23, 24]
right_hand_keep_points = [i for i in range(21)]

face_keep_idx = [face_keep_points[i] for i in range(len(face_keep_points))]
left_hand_keep_idx = [i + 468 for i in left_hand_keep_points]
pose_keep_idx = [i + 468 + 21 for i in pose_keep_points]
right_hand_keep_idx = [i + 468 + 21 + 33 for i in right_hand_keep_points]

landmarks_to_keep = face_keep_idx + left_hand_keep_idx + pose_keep_idx + right_hand_keep_idx
TOTAL_ROWS = 543
NUM_FRAMES = 30
desired_num_rows = len(landmarks_to_keep) * 2

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Recolor Feed
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_COUNTOURS,
                                  mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                                  )

        # Draw right hand landmarks
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                                  )

        # Draw left hand landmarks
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                                  )

        # Draw pose landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        try:
            
       
            # Extract Face landmarks
            face = results.face_landmarks.landmark if results.face_landmarks else []
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            # Extract Right Hand landmarks
            right_hand = results.right_hand_landmarks.landmark if results.right_hand_landmarks else []
            right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())

            # Extract Left Hand landmarks
            left_hand = results.left_hand_landmarks.landmark if results.left_hand_landmarks else []
            left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten())
            
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark if results.pose_landmarks else []
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            # Check if the arrays are empty and handle the case
            if len(face_row) == 0:
                face_row = [0.0] * (len(face_keep_idx) )  # Filling with zeros if no face landmarks
            else:
                face_row = np.array(face_row)[face_keep_idx]

            if len(right_hand_row) == 0:
                right_hand_row = [0.0] * (len(right_hand_keep_idx))  # Filling with zeros if no right hand landmarks
            else:
                right_hand_row = np.array(right_hand_row)[right_hand_keep_idx]
                
            if len(pose_row) == 0:
                pose_row = [0.0] * (len(pose_keep_idx) )  # Filling with zeros if no pose landmarks
            else:
                pose_row = np.array(pose_row)[pose_keep_idx]

            if len(left_hand_row) == 0:
                left_hand_row = [0.0] * (len(left_hand_keep_idx))  # Filling with zeros if no left hand landmarks
            else:
                left_hand_row = np.array(left_hand_row)[left_hand_keep_idx]

            # Concatenate the specific landmark points
            row = np.concatenate([
                face_row,
                right_hand_row,
                pose_row,
                left_hand_row
            ])

            # Reshape and create a DataFrame
            data = pd.DataFrame([row])

            # Adjust the number of frames
            if data.shape[0] > NUM_FRAMES:  # if row = 1 > 30
                indices = np.arange(0, data.shape[0], data.shape[0] // NUM_FRAMES)[:NUM_FRAMES]
                data = data.iloc[indices]
            elif data.shape[0] < NUM_FRAMES:  # padding the videos
                rows = NUM_FRAMES - data.shape[0]
                data = pd.concat([pd.DataFrame(np.zeros((rows, len(row)))), data])

            print(data.shape)

            # Continue with the rest of the processing as needed

        except Exception as e:
            print("Error:", e)
            pass

        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


AttributeError: module 'mediapipe.python.solutions.holistic' has no attribute 'FACEMESH_COUNTOURS'

In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    train_dir: Path
    test_dir: Path

In [4]:
from ASLD_step2.constants import *
from ASLD_step2.utils import read_yaml, create_directories


CONFIG_FILE_PATH = Path("configs/config.yaml")

In [18]:
from ASLD_step2.constants import *
from ASLD_step2.utils import read_yaml,create_directories
from ASLD_step2.entity import DataIngestionConfig
from ASLD_step2.Exception import ASLDException
import sys
import shutil
class Configuration:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH
                ):
        #reading the config.yaml file by importing from constants
        self.config = read_yaml(config_filepath)
        #creating parent directories entity
        create_directories([self.config.artifacts_root])


    """
    this function is written to create the skeletion of DataIngestion entity 
    and will return path for DataIngestionConfig
    """  
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        #read config file to get data_ingestion attributes of DataIngestion
        try:
            config = self.config.data_ingestion
            #create directories for attributes of data_ingestion
            create_directories([config.root_dir])
            create_directories([config.train_dir])
            create_directories([config.test_dir])
            
            #assign the entity with its properities and return it
            data_ingestion_config = DataIngestionConfig(
                root_dir = config.root_dir,
                source_URL = config.source_URL,
                local_data_file = config.local_data_file,
                unzip_dir = config.unzip_dir,
                train_dir = config.train_dir,
                test_dir = config.test_dir
            )

            return data_ingestion_config  
        except Exception as e:
            raise ASLDException(e,sys)


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import shutil

def split_videos(self):
        # Load the CSV file
        csv_file = os.path.join(self.config.unzip_dir,'videos_id.csv')
        df = pd.read_csv(csv_file)
        captions = df['caption'].tolist()  # Get the captions column
        ids = df['ID'].tolist()  # Get the IDs column

        # Create a dictionary mapping IDs to captions
        id_to_caption = dict(zip(ids, captions))

        # List all video files in the video folder
        video_files = [f for f in os.listdir(self.config.unzip_dir) if f.endswith(".mp4")]

        # Split the videos
        train_ratio = 0.80
        videos_train, videos_test = train_test_split(video_files, train_size=train_ratio, random_state=42)

        # Create train and test folders if they don't exist
        train_folder = self.config.train_dir
        test_folder =  self.config.test_dir
        os.makedirs(train_folder, exist_ok=True)
        os.makedirs(test_folder, exist_ok=True)

        # Move videos to train folder and label them
        for video in videos_train:
            video_id = os.path.splitext(video)[0]
            caption = id_to_caption.get(video_id, "Unknown")  # Get the caption for the video ID
            labeled_video = f"{caption}_{video}"
            shutil.move(os.path.join(video_folder, video), os.path.join(train_folder, labeled_video))

        # Move videos to test folder and label them
        for video in videos_test:
            video_id = os.path.splitext(video)[0]
            caption = id_to_caption.get(video_id, "Unknown")  # Get the caption for the video ID
            labeled_video = f"{caption}_{video}"
            shutil.move(os.path.join(video_folder, video), os.path.join(test_folder, labeled_video))

        print("Videos split into train and test sets.")
